import necessary libraries

In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import argparse
from collections import Iterable
import uuid
import os
import json
import urllib.request

/var/folders/zh/9q0t4zld60d2qm1_23q36qwh0000gn/T/ipykernel_22642/1285710457.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [2]:
class Scraper():

    #use selenium to open desired webpage using Chrome
    def __init__(self, url):
        self.driver = webdriver.Chrome()
        self.url = url
        self.page = self.driver.get(self.url)

        #main book categories of the website
        self.book_main_categories = []
        self.category_subcategories = []
        self.books_list_page = []
        self.books_list_slider = []

        #create empty metadata dictionary
        self.metadata_dictionary = {"Unique id" : " ",
                                    "UUID" : " ",
                                    "Book Title" : " ",
                                    "Author" : " ",
                                    "Initial Price" : " ",
                                    "Current Price" : " ",
                                    "ISBN" : " ",
                                    "Number of Pages" : " ",
                                    "Published Date" : " ",
                                    "Publisher" : " ",
                                    "Stock" : " ",
                                    "Availability" : " ",
                                    "Height" : " ",
                                    "Width" : " ",
                                    "Link to image": " ",
                                    "Bookstore Name" : " ",
                                    "Bookstore Address" : " ",
                                    "Schedule" : " ",
                                    "Collection Time" : " "}

        self.list_dict_metadata = []

    #bypass cookies
    def bypass_cookies(self):
    
        # wait so website doesn't suspect you are a bot
        time.sleep(2)

        # if website has cookies
        try:
            accept_cookies_button = self.driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
            accept_cookies_button.click()

        # if website doesn't have cookies 
        except: 
            pass

    # define flags for different conditions under which you can run the scraper
    def scraper_flags(self):
        # first flag is for choosing desired book category
        self.category_parser = argparse.ArgumentParser(description = "Which main book category you want to select?")

        ##for when runing .py file!!
        # f for fiction
        self.category_parser.add_argument('-f', action = "store_true")
        # c for crime
        self.category_parser.add_argument('-c', action = "store_true")
        # sf for science-fiction
        self.category_parser.add_argument('-sf', action = "store_true")
        # g for graphic novels and manga
        self.category_parser.add_argument('-g', action = "store_true")
        # nf for non-fiction
        self.category_parser.add_argument('-nf', action = "store_true")

        ## for now in jupyter notebook use the below
        #self.category_parser.add_argument(category_flag, action = "store_true")

        self.category_args = self.category_parser.parse_args()

    
    #click/access one of the links
    def access_book_category(self, item):
        self.driver.get(item)  

    #retrieves list of link with main books categories
    def get_main_books_categories(self, desired_category):

        #first part of xpath always navigated to desktop version of website desktop-navs
        book_category_path = self.driver.find_element_by_xpath('//div[@class = "navs-container desktop-navs"]/ul[@class = "subnavs"][1]/li[2]')
        # get all a elements containing the book categories
        books_main_list = book_category_path.find_elements_by_xpath('.//span[@class = "name nav-header-link"]/a')
        # we only interested in 5 categories: fiction, crime, science finction, graphic novel and non-fiction
        # leaves out the children book categories
        books_main_list = books_main_list[0:5]

        # make this into a list compression 
        for item in books_main_list:
            #a_tag = item.find_element_by_tag_name('a')
            link = item.get_attribute('href')
            self.book_main_categories.append(link)

        #return book_main_categories
        #based on passed category_flag, return desired link for next method
        ## FOR WHEN RUNING .PY SCRIPT
        #if self.category_args.fiction == True:
         #   return self.book_main_categories[0]
        #elif self.category_args.c == True:
         #   return self.book_main_categories[1]
        #elif self.category_args.sf == True:
         #   return self.book_main_categories[2]
        #elif self.category_args.g == True:
         #   return self.book_main_categories[3]
        #elif self.category_args.nf == True:
         #   return self.book_main_categories[4]

         ## JUST FOR NOW FOR IPYNB FILES
        if desired_category == 'fiction':
            return self.book_main_categories[0]
        elif desired_category == 'crime':
            return self.book_main_categories[1]
        elif desired_category == 'science fiction':
            return self.book_main_categories[2]
        elif desired_category == 'graphic novel':
            return self.book_main_categories[3]
        elif desired_category == 'non fiction':
            return self.book_main_categories[4]



    #from main category - can now access all sub-categories buttons
    def get_category_subcategories(self, desired_category):

        self.category_subcategories = []
        # calls function to retrieve desired book category and then loads corresponding page
        category_header = self.get_main_books_categories(desired_category)
        self.access_book_category(category_header)

        #find list of category sub-divisions
        subcategories_list = self.driver.find_elements_by_xpath('//div[@class = "span3 tablet-span6 mobile-span6"]//a')

        for item in subcategories_list:
            link = item.get_attribute('href')
            self.category_subcategories.append(link)

    
    ## subcategory page: check if 'Our best <subcategory> exist if so you can scroll left/right and press
    ## see all to display all the pages, otherwise skips this steps and directly goes to all pages display
    
    def finds_best_subcategory_list(self):

        #for subcategory in self.category_subcategories :

        # if header and see more button then press this - get data from full pages
        
        header_path = self.driver.find_element_by_xpath('//header[@class = "span12 pages-header-row"]')

            #find see all button - sometimes called see more but same xpath
        self.see_all_button = header_path.find_element_by_xpath('./a[@class = "button button-teal"]')
        self.see_all_button.click()
        time.sleep(3)

        try:
            header_path = self.driver.find_element_by_xpath('//header[@class = "span12 pages-header-row"]')
            self.see_all_button = header_path.find_element_by_xpath('./a[@class = "button button-teal"]')
            self.see_all_button.click()
            time.sleep(3)
        except:
            pass

        #self.get_book_list_page(3)
        
        #if you want instead to only get information from slider
        #self.get_book_list_slider()
                        
        
    ##GET FINAL BOOKS LIST
    
    #navigation method 1: get current url, scroll down, get next page url - FASTEST METHOD
    #returns book list from see more pages
    def get_book_list_page(self, pages_no):
        
        self.books_list_page = []

        if self.driver.current_url[-7 : -1] == "?page=":

            self.current_url = self.driver.current_url[:-7] + '/page/'

            i = 1
            while i <= pages_no:

                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                list_books_links = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in list_books_links:
                    link = item.get_attribute('href')
                    self.books_list_page.append(link)

                i += 1
                self_new_current_url = ''
                self.new_current_url = self.current_url + str(i)
                self.driver.get(self.new_current_url) 

        
        elif self.driver.current_url[-7 : -1] == "/page/":

            self.current_url = self.driver.current_url[:-1]

            i = 1
            while i <= pages_no:

                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                list_books_links = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in list_books_links:
                    link = item.get_attribute('href')
                    self.books_list_page.append(link)

                i += 1
                self_new_current_url = ''
                self.new_current_url = self.current_url + str(i)
                self.driver.get(self.new_current_url)

        else:

            self.current_url = self.driver.current_url + '/page/'
            i = 1
            while i <= pages_no:

                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                list_books_links = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in list_books_links:
                    link = item.get_attribute('href')
                    self.books_list_page.append(link)

                i += 1
                self_new_current_url = ''
                self.new_current_url = self.current_url + str(i)
                self.driver.get(self.new_current_url)
    
    # Method to remove duplicate book links
    def remove_duplicate_book_links(self, list):
     for item in list:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in self.remove_duplicate_book_links(item):
                 yield x
         else:        
             yield item
    
    #def check_key_exist(self, my_dict, my_key, my_value):
     #   if my_key in my_dict:
      #      if not isinstance(my_dict[my_key], list):
       #         my_dict[my_key] = [my_dict[my_key]]
        #    my_dict[my_key].append(my_value)
        #else:
         #   my_dict[my_key] = my_value
    
    #generate unique id for each book link page - is this id user friendly??/ could use ISBN of each book

    def generate_unique_id(self, item):

        self.metadata_dictionary["Unique id"] = item.split('/')[-1]
        
    
    ## loop through book links, access one at a time and then collect the meta-data from each book page
    def collect_book_metadata(self, list):
        
        self.list_dict_metadata = []

        for item in list:

            # access individual book page
            self.access_book_category(item)

            #generate unique ids from page id and uuid
            self.generate_unique_id(item)
            self.metadata_dictionary["UUID"] = str(uuid.uuid4())
            
            # get all the structured metadata

            book_title = self.driver.find_element_by_xpath('//span[@class = "book-title"]').text
            self.metadata_dictionary["Book Title"] = book_title

            author = self.driver.find_element_by_xpath('//span[@itemprop = "author"]').text
            self.metadata_dictionary["Author"] = author
            
            # for coming soon category 
            try:
                initial_price = self.driver.find_element_by_xpath('//b[@class = "price-rrp"]').text
                no_pages = self.driver.find_element_by_xpath('//span[@itemprop = "numberOfPages"]').text
                stock = self.driver.find_element_by_xpath('//span[@id = "scope_offer_availability"]').text
                availability = self.driver.find_element_by_xpath('//p[@class = "stock-message"]').text

            except:
                initial_price = 'NaN'
                no_pages = 'NaN'
                stock = 'NaN'

            self.metadata_dictionary["Initial Price"] = initial_price
            self.metadata_dictionary["Number of Pages"] = no_pages
            self.metadata_dictionary["Stock"] = stock
            self.metadata_dictionary["Availability"] = availability

            current_price = self.driver.find_element_by_xpath('//b[@itemprop = "price"]').text
            self.metadata_dictionary["Current Price"] = current_price
                
            published_date = (self.driver.find_element_by_xpath('//meta[@itemprop = "datePublished"]')).get_attribute('content')
            self.metadata_dictionary["Published Date"] = published_date

            publisher = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[1]/span').get_attribute("innerHTML")
            self.metadata_dictionary["Publisher"] = publisher

            isbn = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[2]/span').get_attribute("innerHTML")
            self.metadata_dictionary["ISBN"] = isbn

            #if dimensions etc try [4] /except [5]
            if len(self.driver.find_elements_by_xpath('//p[@class = "spec"]/i')) == 6:
                height = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[3]/span/span[1]').get_attribute("innerHTML")
                width = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[3]/span/span[2]').get_attribute("innerHTML")    

            elif len(self.driver.find_elements_by_xpath('//p[@class = "spec"]/i')) == 8:
                height = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[4]/span/span[1]').get_attribute("innerHTML")
                width = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[4]/span/span[2]').get_attribute("innerHTML")

            elif len(self.driver.find_elements_by_xpath('//p[@class = "spec"]/i')) == 10:
                # for normal case scenario
                
                if self.driver.find_element_by_xpath('//p[@class = "spec"]/i[5]').get_attribute("innerHTML")[0:9] == 'Dimension':

                    height = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[5]/span/span[1]').get_attribute("innerHTML")
                    width = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[5]/span/span[2]').get_attribute("innerHTML")
                  
                elif self.driver.find_element_by_xpath('//p[@class = "spec"]/i[5]').get_attribute("innerHTML")[0:7] == 'Edition':

                    height = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[4]/span/span[1]').get_attribute("innerHTML")
                    width = self.driver.find_element_by_xpath('//p[@class = "spec"]/i[4]/span/span[2]').get_attribute("innerHTML")
            
            self.metadata_dictionary["Height"] = height
            self.metadata_dictionary["Width"] = width
            
            # get the image links/unstructured data
            image_links = self.driver.find_element_by_xpath('//div[@class = "book-image-main"]//img').get_attribute("src")
            self.metadata_dictionary["Link to image"] = image_links

            #find the click & collect metadata
            try:
                self.click_and_collect()
            except:
                pass

            time.sleep(4)
            try:
                bookstore_name = self.driver.find_element_by_xpath('//div[@class = "store"][1]//div[@class = "title"]').get_attribute("innerHTML")

                bookstore_address = self.driver.find_element_by_xpath('//div[@class = "store"][1]//div[@class = "address"]').get_attribute("innerHTML")

                bookstore_schedule = self.driver.find_element_by_xpath('//div[@class = "store"][1]//div[@class = "hours"]').get_attribute("innerHTML")

                collection_time = self.driver.find_element_by_xpath('//div[@class = "store"][1]//div[4]').get_attribute("innerHTML")
            
            except NoSuchElementException:
                bookstore_name = 'Book is not out yet'
                bookstore_address = 'Book is not out yet'
                bookstore_schedule = 'Book is not out yet'
                collection_time = 'Book is not out yet'


            self.metadata_dictionary["Bookstore Name"] = bookstore_name
            self.metadata_dictionary["Bookstore Address"] = bookstore_address
            self.metadata_dictionary["Collection Time"] = collection_time
            
            self.metadata_dictionary["Schedule"] = bookstore_schedule
            #if book is not available for collection today
            if self.metadata_dictionary["Schedule"] == '':
                self.metadata_dictionary["Schedule"] = "Book not available for collection today"
            

        # now append dictionary to a list of dictionaries
            dictionary_copy = self.metadata_dictionary.copy()
            self.list_dict_metadata.append(dictionary_copy)       
            
    ## first method: enter book and navigate to click & collect information
    # will need to adapt it to loop through all books
    def click_and_collect(self):
        #self.access_book_category(self.all_links_next_page, index)

        click_and_collect_button = self.driver.find_element_by_xpath('//div[@class = "book-actions"]//button[@class = "button button-gold js-open-modal"]')
        click_and_collect_button.click()

        search_bar = self.driver.find_element_by_xpath('//input[@placeholder = "Town, city, or postcode"]')
        time.sleep(2)
        search_bar.send_keys("WC1 0RW")

        go_button = self.driver.find_element_by_xpath('//button[@id = "searchterm"]')
        go_button.click()


    #create folders for the data to be saved in
    def create_raw_data_folder(self):

        root_directory = '/Users/maya/Desktop/AiCore_git/Data_Collection_Project'
        raw_data_directory =  os.path.join(root_directory, 'raw_data')

        if root_directory == os.getcwd():

            if os.path.exists(raw_data_directory):
                os.chdir(raw_data_directory)
            else:
                os.mkdir('raw_data')
                os.chdir(raw_data_directory)
        else:
            os.chdir(raw_data_directory)

    def create_category_folders(self, category):
        
        self.create_raw_data_folder()
        current_directory = os.getcwd()
        category_directory = os.path.join(current_directory, category)

        if current_directory:

            if os.path.exists(category_directory):
                os.chdir(category_directory)
            else:
                os.mkdir(category)
                os.chdir(category_directory)
        else:
            os.chdir(category_directory)


    def create_subcategory_folders(self, subcategory):

        #self.create_category_folders(category)
        current_directory = os.getcwd()
        subcategory_directory = os.path.join(current_directory, subcategory)

        if os.path.exists(subcategory_directory):
            os.chdir(subcategory_directory)
        else:
            os.mkdir(subcategory)
            os.chdir(subcategory_directory)
        
        self.saving_directory = subcategory_directory
    
    def save_book_covers(self):
        for image in range(len(self.list_dict_metadata)):

            image_url = self.list_dict_metadata[image]["Link to image"]
            save_path = os.path.join(os.getcwd(), (str(image) + '.jpg'))
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'MyApp/1.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(image_url, save_path)


    

#### also removes duplicate links

In [9]:
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

#### if name == main run

In [3]:
if __name__ == "__main__":

    waterstones = Scraper("https://www.waterstones.com")
    waterstones.bypass_cookies()

    #list_categories = ["fiction", "crime", "science fiction", "graphic novel", "non fiction"]
    list_categories = ["fiction"]
    #final_book_list = []


    for category in list_categories:
        
        #create folders for each category
        waterstones.create_category_folders(category)

        #returns list of sub-categories
        waterstones.get_category_subcategories(category)

        for subcategory in waterstones.category_subcategories[0:1]:
            
            #creates folder for current subcategory
            subcategory_folder = subcategory.split('/')[-1]
            waterstones.create_subcategory_folders(subcategory_folder)

            # access individual page, scrolls through and retrieves links for books
            final_book_list = []
            waterstones.access_book_category(subcategory)
            waterstones.finds_best_subcategory_list()
            waterstones.get_book_list_page(1)
            #final_book_list.append(waterstones.books_list_page)
            final_book_list = waterstones.books_list_page
            
           # list without duplicate book links
            #currated_list = []
            currated_list = list(waterstones.remove_duplicate_book_links(final_book_list))

           #collects all the metadta
            waterstones.collect_book_metadata(currated_list)

            # saves list of dictionary metadata as json file
            with open(os.path.join(waterstones.saving_directory, 'data.json'), 'w') as folder:
                json.dump(waterstones.list_dict_metadata, folder)

            # create image folder to save all book covers for this sub-category
            if os.path.exists(os.path.join(waterstones.saving_directory, 'images')):
                os.chdir(os.path.join(waterstones.saving_directory, 'images'))
            else:
                os.mkdir('images')
                os.chdir(os.path.join(waterstones.saving_directory, 'images'))

            #save images
            waterstones.save_book_covers()

            # go back to category folder (first exist images subfolder then goes back)
            os.chdir('../..')
        
        #go back to raw-data folder to create new category folder on next iteration
        os.chdir('..')
        


extra code from initial scraper

In [117]:
   #method 1: scroll down to end of page, wait, press load more and so on (number rep = number of pages scrolled)
    #inside this method later will need to also get list of all links to the books -SECOND PLACE SPEED
    def navigation_method_1(self, pages_no):

        scroll_pause_time = 2
        #gets current page height
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        #here using while true loop it go on forever - for now add counter to only load first 5 pages
        #some problem with page number!! only loads until page 4 but no further?
        i = 0
        while i <= pages_no:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            new_height = self.driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:

                all_books = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in all_books:
                    link = item.get_attribute('href')
                    self.all_links_page.append(link)
                show_more_button = self.driver.find_element_by_xpath('//button[@class = "button button-teal"]')
                show_more_button.click()
                break

            #once at the bottom of page and see more pressed- reinitialises height
            last_height = new_height
            i +=1
    
    #method 2: scroll down, scroll back up and then press next arrow to navigate to next page -SLOWEST
    def navigation_method_2(self, pages_no):

        scroll_pause_time = 2

        i = 1
        while i <= pages_no:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            all_books_this_page = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
            for item in all_books_this_page:
                link = item.get_attribute('href')
                self.all_links_next_page.append(link)

            #time.sleep(scroll_pause_time)

            try:
                go_back_up_button = self.driver.find_element_by_xpath('//div[@class = "backtotop show stick"]')
                go_back_up_button.click()
            except:
                go_back_up_button = self.driver.find_element_by_xpath('//div[@class = "backtotop show"]')
                go_back_up_button.click()

            time.sleep(scroll_pause_time)

            next_page = self.driver.find_element_by_xpath('//a[@title = "Go to next page"]')
            next_page.click()
            i += 1
            #break

IndentationError: unexpected indent (2578035342.py, line 3)

In [ ]:

    #alternatively can get directly a link to all existent sub-categories of all main categories
    def get_all_subcategories(self):
        all_subcategories = self.driver.find_elements_by_xpath('//div[@class = "navs-container desktop-navs"]//li[@class = "subnav"][2]//li[@class = "subnav-item"]')

        for item in all_subcategories:
            a_tag = item.find_element_by_tag_name('a')
            link = a_tag.get_attribute('href')
            self.all_subcategories.append(link)

In [ ]:
  
    # method 2 returning book list from slider bar
    def get_book_list_slider(self):

        swiper_path = self.driver.find_element_by_xpath('//div[@class = "swiper-wrapper"]')
        titles_path = swiper_path.find_elements_by_xpath('.//div[@class = "title-wrap"]')

        for item in titles_path:
            a_tag = item.find_element_by_tag_name('a')
            link = a_tag.get_attribute('href')
            self.books_list_slider.append(link)